In [1]:
from RandomTesting.__RandomTesting__ import Random_Testing, calculate_extract_loss
from tqdm import tqdm
from sklearn import neighbors
from pykrige.ok import OrdinaryKriging
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
import pandas as pd
import gc

warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = ""
tf.config.optimizer.set_jit(True)

# AIR_Kriging

In [ ]:
# 2020/12/03 OK
air1718 = pd.read_csv('air-201718.csv')
air1718_arr = air1718[7075495:]
air19 = pd.read_csv('air-2019.csv')
air171819_arr = np.concatenate([air1718_arr,air19])

first_lst = []
count=0
for data_nb in tqdm(range(len(air171819_arr))):
    if (air171819_arr[data_nb][0][-8:] == '08:00:00') or (air171819_arr[data_nb][0][-8:] == '20:00:00'):
        count+=1
        first_lst.append(air171819_arr[data_nb])
print(np.array(first_lst).shape)
df = pd.DataFrame(data=np.array(first_lst),columns=['dt',"lat", "lon",'pm25','pm10','temperature','humidity'])
df.to_csv('AIR_Training.csv',index=False)

lat_min = 21.87; lat_max = 25.35
lon_min = 120; lon_max = 122.04
pm25_max = 300


df = pd.read_csv('AIR_Training.csv')
df_lat_ok = df[(df['lat'] > lat_min)][df[(df['lat'] > lat_min)]['lat'] < lat_max]
df_lat_lon_ok = df_lat_ok[(df_lat_ok['lon'] > lon_min)][df_lat_ok[(df_lat_ok['lon'] > lon_min)]['lon'] < lon_max]
df_finall = df_lat_lon_ok[df_lat_lon_ok['pm25'] < pm25_max]

mou = pd.read_csv('mou_process_done.csv')
del mou['Unnamed: 0']

dts=df_finall.dt.unique()

air_kriging = []
grid_lon = np.array(np.arange(204),dtype='float64')
grid_lat = np.array(np.arange(348),dtype='float64') 

for dt in tqdm(dts[:]):
    one_dt=df_finall[df_finall.dt==dt]

    x_train = one_dt[['lat', 'lon']]
    y_train = one_dt.iloc[:, 3:4] #pm2.5
    print(len(x_train))
    matrix_lon_list = [];matrix_lat_list = []; value_list=[]
    for i in range(len(x_train)):
        matrix_lon = int(round((np.array(x_train)[i][1]-120)/((122.04-120)/204),0))
        matrix_lat = int(round((np.array(x_train)[i][0]-21.87)/((25.35-21.87)/348),0))
        matrix_lon_list.append(matrix_lon)
        matrix_lat_list.append(matrix_lat)
        value_list.append(np.array(y_train)[i][0])

    matrix_lon_array = np.concatenate([np.array(matrix_lon_list), mou['matrix_lon'].values])
    matrix_lat_array = np.concatenate([np.array(matrix_lat_list), mou['matrix_lat'].values])
    train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T)
    
    mon_value = np.zeros_like(mou['matrix_lon'].values)
    train_y = np.concatenate([np.array(value_list),mon_value])

    OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear', weight = True,
                         exact_values = False, verbose=False)

    values, ss1 = OK.execute('grid', grid_lon, grid_lat)
    air_kriging.append(values)

    plt.imshow(values,origin='lower')
    plt.title(str(dt))
    plt.colorbar()
    plt.show()
    
    del matrix_lon_array; del matrix_lat_array; del train_x; del train_y; del OK; del values; del ss1        

print(len(air_kriging))

# find error by Manual inspection
air_kriging = np.delete(air_kriging,[19,20,28,57,77,170,193,513,514,551,623,634,651,688,1215,
                                     1223,1252,1254,1266,1267,1268,1271,1284,1303,1308,1353,
                                     1359,1371,1375,1445],axis=0)

np.save('AIR_DomainB_Training',np.reshape(air_kriging,(1597,348,204,1)))

In [ ]:
# 2020/12/03 OK

air20_arr = np.array(pd.read_csv('air-2020.csv'))
first_lst = []
count=0
for data_nb in tqdm(range(len(air20_arr))):
    if (air20_arr[data_nb][0][-8:] == '08:00:00') or (air20_arr[data_nb][0][-8:] == '20:00:00'):
        count+=1
        first_lst.append(air20_arr[data_nb])
print(np.array(first_lst).shape)
df = pd.DataFrame(data=np.array(first_lst),columns=['dt',"lat", "lon",'pm25','pm10','temperature','humidity'])
df.to_csv('AIR_Testing.csv',index=False)


lat_min = 21.87; lat_max = 25.35
lon_min = 120; lon_max = 122.04
pm25_max = 300

df = pd.read_csv('AIR_Testing.csv')
df_lat_ok = df[(df['lat'] > lat_min)][df[(df['lat'] > lat_min)]['lat'] < lat_max]
df_lat_lon_ok = df_lat_ok[(df_lat_ok['lon'] > lon_min)][df_lat_ok[(df_lat_ok['lon'] > lon_min)]['lon'] < lon_max]
df_finall = df_lat_lon_ok[df_lat_lon_ok['pm25'] < pm25_max]

mou = pd.read_csv('mou_process_done.csv')
del mou['Unnamed: 0']

dts=df_finall.dt.unique()

air_kriging = []
grid_lon = np.array(np.arange(204),dtype='float64')
grid_lat = np.array(np.arange(348),dtype='float64') 

for dt in tqdm(dts[:]):
    one_dt=df_finall[df_finall.dt==dt]

    x_train = one_dt[['lat', 'lon']]
    y_train = one_dt.iloc[:, 3:4] #pm2.5
    print(len(x_train))
    matrix_lon_list = [];matrix_lat_list = []; value_list=[]
    for i in range(len(x_train)):
        matrix_lon = int(round((np.array(x_train)[i][1]-120)/((122.04-120)/204),0))
        matrix_lat = int(round((np.array(x_train)[i][0]-21.87)/((25.35-21.87)/348),0))
        matrix_lon_list.append(matrix_lon)
        matrix_lat_list.append(matrix_lat)
        value_list.append(np.array(y_train)[i][0])

    matrix_lon_array = np.concatenate([np.array(matrix_lon_list), mou['matrix_lon'].values])
    matrix_lat_array = np.concatenate([np.array(matrix_lat_list), mou['matrix_lat'].values])
    train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T)
    
    mon_value = np.zeros_like(mou['matrix_lon'].values)
    train_y = np.concatenate([np.array(value_list),mon_value])

    OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear', weight = True,
                         exact_values = False, verbose=False)

    values, ss1 = OK.execute('grid', grid_lon, grid_lat)
    air_kriging.append(values)

    plt.imshow(values,origin='lower')
    plt.title(str(dt))
    plt.colorbar()
    plt.show()
    
    del matrix_lon_array; del matrix_lat_array; del train_x; del train_y; del OK; del values; del ss1        

print(len(air_kriging))

# find error by Manual inspection
air_kriging = np.delete(air_kriging,[161,162,164,206,210,236,260,272,280,
                                     281,282,288,289,291,292,298,304,312,
                                     315,318,329,332,346,356],axis=0)

np.save('AIR_DomainB_Testing',np.reshape(air_kriging,(338,348,204,1)))

In [ ]:
# 2020/12/03 OK

AIR_Training = np.load('AIR_DomainB_Training.npy')
AIR_Testing = np.load('AIR_DomainB_Testing.npy')
sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

AIR73_Training = np.zeros_like(AIR_Training)
AIR73_Testing = np.zeros_like(AIR_Testing)

for lat, lon in zip(sta_lat, sta_lon):
    AIR73_Training[:, lat, lon, :] = AIR_Training[:, lat, lon, :]
    AIR73_Testing[:, lat, lon, :] = AIR_Testing[:, lat, lon, :]

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

mou = pd.read_csv('mou_process_done.csv')
mou_lat = [mou['matrix_lat'][0], mou['matrix_lat'][71], mou['matrix_lat'][92]] #玉山, 馬比杉山, 北大武山
mou_lon = [mou['matrix_lon'][0], mou['matrix_lon'][71], mou['matrix_lon'][92]]
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = AIR73_Training


for i in tqdm(range(input_data.shape[0])):
    value_list, result = ([] for _ in range(2))
      
    for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])

        
    lat_all_mou = np.concatenate([sta_lat, mou_lat])
    lon_all_mou = np.concatenate([sta_lon, mou_lon])
    train_x = np.vstack((lat_all_mou,lon_all_mou)).T
    
    mon_value = np.zeros_like(mou_lat)
    value_list_mou = np.concatenate([value_list,mon_value])
    train_y = value_list_mou

    knn = neighbors.KNeighborsRegressor(5, weights = 'distance') 
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):result.append(test_y[k*204:(k+1)*204])   
    total_result.append(result)

AIR73_train_fill = total_result
np.save('AIR_DomainA_Training_K5',np.reshape(np.array(AIR73_train_fill),(1597,348,204,1)))

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

mou = pd.read_csv('mou_process_done.csv')
mou_lat = [mou['matrix_lat'][0], mou['matrix_lat'][71], mou['matrix_lat'][92]] #玉山, 馬比杉山, 北大武山
mou_lon = [mou['matrix_lon'][0], mou['matrix_lon'][71], mou['matrix_lon'][92]]
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = AIR73_Testing


for i in tqdm(range(input_data.shape[0])):
    value_list, result = ([] for _ in range(2))

    for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])
    
    lat_all_mou = np.concatenate([sta_lat, mou_lat])
    lon_all_mou = np.concatenate([sta_lon, mou_lon])
    train_x = np.vstack((lat_all_mou,lon_all_mou)).T
    
    mon_value = np.zeros_like(mou_lat)
    value_list_mou = np.concatenate([value_list,mon_value])
    train_y = value_list_mou

    knn = neighbors.KNeighborsRegressor(5, weights = 'distance') 
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):result.append(test_y[k*204:(k+1)*204])   
    total_result.append(result)

AIR73_train_fill = total_result
np.save('AIR_DomainA_Testing_K5',np.reshape(np.array(AIR73_train_fill),(338,348,204,1)))

# SAT_Align

In [ ]:
# 2020/12/03 OK

arrange_pd_training = pd.read_csv('AIR_Training.csv')
SAT_time_training = np.load('Satellite_29X17_data/SAT_pm25/an_training_time.npy',allow_pickle=True)
SAT_training = np.load('SAT_interlinear_Training_348X204_PM25.npy',mmap_mode='r')

l1 = list(np.array(SAT_time_training,dtype='str'))
l2 = arrange_pd_training.dt.unique()
l3 = [x for x in l1 if x not in l2]
print(l3)
align_SAT = []

for i in range(len(l1)):
    if l1[i] in l3:
        print(i)
    else:
        align_SAT.append(SAT_training[i])
print('total:',len(align_SAT))

Sat_train_del = np.delete(np.array(align_SAT),[19,20,28,57,77,170,193,513,514,551,623,634,651,688,1215,
                                               1223,1252,1254,1266,1267,1268,1271,1284,1303,1308,1353,
                                               1359,1371,1375,1445],axis=0)
print(Sat_train_del.shape)
np.save('SAT_DomainB_Training.npy',Sat_train_del)

In [ ]:
# 2020/12/03 OK
arrange_pd = pd.read_csv('AIR_Testing.csv')
SAT_time_testing = np.load('Satellite_29X17_data/SAT_pm25/an_testing_time.npy',allow_pickle=True)
SAT_testing = np.load('SAT_interlinear_Testing_348X204_PM25.npy',mmap_mode='r')

l1 = list(np.array(SAT_time_testing,dtype='str'))
l2 = arrange_pd.dt.unique()

l3 = [x for x in l1 if x not in l2]
print(l3)

align_SAT = []

for i in range(len(l1)):
    if l1[i] in l3:
        print(i)
    else:
        align_SAT.append(SAT_testing[i])
        
Sat_test_del = np.delete(align_SAT,[161,162,164,206,210,236,260,272,280,
                                     281,282,288,289,291,292,298,304,312,
                                     315,318,329,332,346,356],axis=0)
print('total:',len(Sat_test_del))
# np.save('SAT_DomainB_Testing',Sat_test_del)

In [ ]:
# 2020/12/03 OK

SAT_Training = np.load('SAT_DomainB_Training.npy')
SAT_Testing = np.load('SAT_DomainB_Testing.npy')

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

SAT73_Training = np.zeros_like(SAT_Training)
SAT73_Testing = np.zeros_like(SAT_Testing)

for lat, lon in zip(sta_lat, sta_lon):
    SAT73_Training[:, lat, lon, :] = SAT_Training[:, lat, lon, :]
    SAT73_Testing[:, lat, lon, :] = SAT_Testing[:, lat, lon, :]

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

mou = pd.read_csv('mou_process_done.csv')
mou_lat = [mou['matrix_lat'][0], mou['matrix_lat'][71], mou['matrix_lat'][92]] #玉山, 馬比杉山, 北大武山
mou_lon = [mou['matrix_lon'][0], mou['matrix_lon'][71], mou['matrix_lon'][92]]
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = SAT73_Training


for i in tqdm(range(input_data.shape[0])):
    value_list, result = ([] for _ in range(2))
    
    for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])

    lat_all_mou = np.concatenate([sta_lat, mou_lat])
    lon_all_mou = np.concatenate([sta_lon, mou_lon])
    train_x = np.vstack((lat_all_mou,lon_all_mou)).T
    
    mon_value = np.zeros_like(mou_lat)
    value_list_mou = np.concatenate([value_list,mon_value])
    train_y = value_list_mou
    
    knn = neighbors.KNeighborsRegressor(5, weights = 'distance')    
    
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):result.append(test_y[k*204:(k+1)*204])
        
    total_result.append(result)

Sat73_train_fill = total_result
np.save('SAT_DomainA_Training_K5',np.reshape(np.array(Sat73_train_fill),(1597,348,204,1)))

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

mou = pd.read_csv('mou_process_done.csv')
mou_lat = [mou['matrix_lat'][0], mou['matrix_lat'][71], mou['matrix_lat'][92]] #玉山, 馬比杉山, 北大武山
mou_lon = [mou['matrix_lon'][0], mou['matrix_lon'][71], mou['matrix_lon'][92]]
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = SAT73_Testing


for i in tqdm(range(input_data.shape[0])):
    value_list, result = ([] for _ in range(2))
    
    for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])

    lat_all_mou = np.concatenate([sta_lat, mou_lat])
    lon_all_mou = np.concatenate([sta_lon, mou_lon])
    train_x = np.vstack((lat_all_mou,lon_all_mou)).T
    
    mon_value = np.zeros_like(mou_lat)
    value_list_mou = np.concatenate([value_list,mon_value])
    train_y = value_list_mou
    
    knn = neighbors.KNeighborsRegressor(5, weights = 'distance')    
    
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):result.append(test_y[k*204:(k+1)*204])
        
    total_result.append(result)

Sat73_train_fill = total_result
np.save('SAT_DomainA_Testing_K5',np.reshape(np.array(Sat73_train_fill),(338,348,204,1)))

# EPA_Align

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]


arrange_pd_training = pd.read_csv('AIR_Training.csv')
SAT_time_training = np.load('Satellite_29X17_data/SAT_pm25/an_training_time.npy',allow_pickle=True)
EPA_training = np.load('EPA73_Training_348X204_PM25.npy',mmap_mode='r')
mou = pd.read_csv('mou_process_done.csv')
del mou['Unnamed: 0']

l1 = list(np.array(SAT_time_training,dtype='str'))
l2 = arrange_pd_training.dt.unique()
l3 = [x for x in l1 if x not in l2]
print(l3)
align_EPA = []

for i in range(len(l1)):
    if l1[i] in l3:
        print(i)
    else:
        align_EPA.append(EPA_training[i])
print('total:',len(align_EPA))

EPA_train_del = np.delete(np.array(align_EPA),[19,20,28,57,77,170,193,513,514,551,623,634,651,688,1215,
                                               1223,1252,1254,1266,1267,1268,1271,1284,1303,1308,1353,
                                               1359,1371,1375,1445],axis=0)
print(EPA_train_del.shape)


# 2020/12/03 OK

EPA_kriging = []
grid_lon = np.array(np.arange(204),dtype='float64')
grid_lat = np.array(np.arange(348),dtype='float64') 

input_data = EPA_train_del

for i in tqdm(range(input_data.shape[0])):
    value_list, result = ([] for _ in range(2))
    for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])

    matrix_lon_array = np.concatenate([np.array(sta_lon), mou['matrix_lon'].values])
    matrix_lat_array = np.concatenate([np.array(sta_lat), mou['matrix_lat'].values])
    train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T)
    
    mon_value = np.zeros_like(mou['matrix_lon'].values)
    train_y = np.concatenate([np.array(value_list),mon_value])

    OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear', weight = True,
                         exact_values = False, verbose=False)

    values, ss1 = OK.execute('grid', grid_lon, grid_lat)
    EPA_kriging.append(values)
np.save('EPA_DomainB_Training',np.reshape(EPA_kriging,(1597,348,204,1)))

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

EPA_Testing = np.load('EPA73_Testing_348X204_PM25.npy',mmap_mode='r')
mou = pd.read_csv('mou_process_done.csv')
del mou['Unnamed: 0']
EPA_kriging = []
grid_lon = np.array(np.arange(204),dtype='float64')
grid_lat = np.array(np.arange(348),dtype='float64') 

input_data = EPA_Testing

for i in tqdm(range(input_data.shape[0])):
    value_list, result = ([] for _ in range(2))
    

    for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])

    matrix_lon_array = np.concatenate([np.array(sta_lon), mou['matrix_lon'].values])
    matrix_lat_array = np.concatenate([np.array(sta_lat), mou['matrix_lat'].values])
    train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T)
    
    mon_value = np.zeros_like(mou['matrix_lon'].values)
    train_y = np.concatenate([np.array(value_list),mon_value])
    
    OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear', weight = True,
                         exact_values = False, verbose=False)

    values, ss1 = OK.execute('grid', grid_lon, grid_lat)
    EPA_kriging.append(values)
np.save('EPA_DomainB_Testing',np.reshape(EPA_kriging,(3000,348,204,1)))

In [ ]:
# 2020/12/03 OK

EPA_Training = np.load('EPA_DomainB_Training.npy')
EPA_Testing = np.load('EPA_DomainB_Testing.npy')
EPA73_Training = np.zeros_like(EPA_Training)
EPA73_Testing = np.zeros_like(EPA_Testing)

for lat, lon in zip(sta_lat, sta_lon):
    EPA73_Training[:, lat, lon, :] = EPA_Training[:, lat, lon, :]
    EPA73_Testing[:, lat, lon, :] = EPA_Testing[:, lat, lon, :]

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

mou = pd.read_csv('mou_process_done.csv')
mou_lat = [mou['matrix_lat'][0], mou['matrix_lat'][71], mou['matrix_lat'][92]] #玉山, 馬比杉山, 北大武山
mou_lon = [mou['matrix_lon'][0], mou['matrix_lon'][71], mou['matrix_lon'][92]]
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = EPA73_Training

for i in tqdm(range(input_data.shape[0])):
    value_list, result = ([] for _ in range(2))
    
    for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])

    lat_all_mou = np.concatenate([sta_lat, mou_lat])
    lon_all_mou = np.concatenate([sta_lon, mou_lon])
    train_x = np.vstack((lat_all_mou,lon_all_mou)).T
    
    mon_value = np.zeros_like(mou_lat)
    value_list_mou = np.concatenate([value_list,mon_value])
    train_y = value_list_mou
    
    knn = neighbors.KNeighborsRegressor(3, weights = 'distance')    
    
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):result.append(test_y[k*204:(k+1)*204])
        
    total_result.append(result)

EPA73_train_fill = total_result
np.save('EPA_DomainA_Training',np.reshape(EPA73_train_fill,(1597,348,204,1)))

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

mou = pd.read_csv('mou_process_done.csv')
mou_lat = [mou['matrix_lat'][0], mou['matrix_lat'][71], mou['matrix_lat'][92]] #玉山, 馬比杉山, 北大武山
mou_lon = [mou['matrix_lon'][0], mou['matrix_lon'][71], mou['matrix_lon'][92]]
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = EPA73_Testing

for i in tqdm(range(input_data.shape[0])):
    value_list, result = ([] for _ in range(2))

    for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])

    lat_all_mou = np.concatenate([sta_lat, mou_lat])
    lon_all_mou = np.concatenate([sta_lon, mou_lon])
    train_x = np.vstack((lat_all_mou,lon_all_mou)).T
    
    mon_value = np.zeros_like(mou_lat)
    value_list_mou = np.concatenate([value_list,mon_value])
    train_y = value_list_mou
    
    knn = neighbors.KNeighborsRegressor(3, weights = 'distance')    
    
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):result.append(test_y[k*204:(k+1)*204])
        
    total_result.append(result)

EPA73_test_fill = total_result
np.save('EPA_DomainA_Testing',np.reshape(EPA73_test_fill,(3000,348,204,1)))

# KNN Random Testing

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

EPA73_test_pm25 = np.load('EPA73_Testing_348X204_PM25.npy', mmap_mode='r')
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')
mou = pd.read_csv('mou_process_done.csv')
mou_lat = [mou['matrix_lat'][0], mou['matrix_lat'][71], mou['matrix_lat'][92]] #玉山, 馬比杉山, 北大武山
mou_lon = [mou['matrix_lon'][0], mou['matrix_lon'][71], mou['matrix_lon'][92]]

K_nb = 5
for extract_nb in range(3,4):
    total_mse, EPA73_test_fill_ex20 = ([] for _ in range(2))
    for qq in range(30):
        matrix20,lats_ex20,lons_ex20=Random_Testing(extract_nb, qq, station_coordinate, EPA73_test_pm25, 
                                                    ex5_lst, ex10_lst, ex15_lst, ex20_lst)

        total_result = []
        test_x = ([[i,j] for i in range(348) for j in range(204)])
        input_data = matrix20
        for i in tqdm(range(input_data.shape[0])):
            value_list, result = ([] for _ in range(2))

            for lat, lon in zip(sta_lat, sta_lon): value_list.append(input_data[i,lat,lon,0])

            matrix_lon_array = np.concatenate([np.array(sta_lon),mou_lon])
            matrix_lat_array = np.concatenate([np.array(sta_lat),mou_lat])

            train_x = np.array(np.vstack((matrix_lat_array,matrix_lon_array)).T)

            mon_value = np.zeros_like(mou_lat)
            train_y = np.concatenate([np.array(value_list),mon_value])        

            knn = neighbors.KNeighborsRegressor(K_nb, weights = 'distance')

            knn_fit = knn.fit(train_x, train_y)
            test_y = knn_fit.predict(test_x)
            for k in range(348): result.append(test_y[k*204:(k+1)*204])
            total_result.append(result)

            del result, test_y
            gc.collect()
#         mse,mae,mape = calculate_extract_loss(np.array(total_result),EPA73_test_pm25[:,:,:,0],lats_ex20,lons_ex20)
#         total_mse.append(mse)
#         print(mse)        
        EPA73_test_fill_ex20.append(total_result)
        del input_data, matrix20, total_result
#     print(np.array(total_mse).mean())        
    np.save('EPA73_testing_KNN'+str(K_nb)+'fill_ex'+str(extract_nb*5),EPA73_test_fill_ex20)
    print('*'*50)    

# Kriging Random Testing 

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

EPA73_test_pm25 = np.load('EPA73_Testing_348X204_PM25.npy', mmap_mode='r')
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')
mou = pd.read_csv('mou_process_done.csv')
mou_lat = [mou['matrix_lat'][0], mou['matrix_lat'][71], mou['matrix_lat'][92]] #玉山, 馬比杉山, 北大武山
mou_lon = [mou['matrix_lon'][0], mou['matrix_lon'][71], mou['matrix_lon'][92]]

grid_lon = np.array(np.arange(204),dtype='float64')
grid_lat = np.array(np.arange(348),dtype='float64') 

total_mse = []
EPA73_testing_Krigingfill_ex20=[]
for qq in range(30):
    matrix20,lats_ex20,lons_ex20=Random_Testing(2,qq,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    total_result = []
    for i in tqdm(range(matrix20.shape[0])):
        value_list = [] 

        for lat, lon in zip(sta_lat, sta_lon): value_list.append(matrix20[i,lat,lon,0])

        matrix_lon_array = np.concatenate([np.array(sta_lon),mou_lat])
        matrix_lat_array = np.concatenate([np.array(sta_lat),mou_lon])
        
        train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T)

        mon_value = np.zeros_like(mou_lat)
        train_y = np.concatenate([np.array(value_list),mon_value])         


        OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear', 
                             weight = True, exact_values = False, verbose=False)

        values, ss1 = OK.execute('grid', grid_lon, grid_lat)
        total_result.append(values)
        del values, ss1, matrix_lon_array, matrix_lat_array, train_x, train_y
        
    mse,mae,mape = calculate_extract_loss(np.array(total_result),EPA73_test_pm25[:,:,:,0],lats_ex20,lons_ex20)
    del matrix20

    total_mse.append(mse)
    print(mse)        

    EPA73_testing_Krigingfill_ex20.append(total_result)
print(np.array(total_mse).mean())
# np.save('EPA73_testing_Krigingfill_ex20',EPA73_testing_Krigingfill_ex20)